# AgentCore Travel Assistant - Memory Demo

Interactive demonstration of cross-session memory with Amazon Bedrock AgentCore Runtime. This notebook shows how the deployed agent maintains context across sessions using persistent memory.

## What You'll Learn

- Invoke a deployed AgentCore agent programmatically
- Test short-term memory within a session
- Test long-term memory across different sessions
- Send multimodal content (images and videos)
- Understand session and user ID management

## Prerequisites

- Deployed AgentCore agent (see `../deployment/`)
- Agent ARN from deployment
- AWS credentials configured
- Python 3.10+ with boto3 installed

## Setup

### Install Dependencies

In [ ]:
!pip install boto3 -q

### Import Libraries

In [ ]:
import boto3
import json
import uuid
import base64
import os
from datetime import datetime

print("✅ All imports successful!")

## Configuration

**Important**: Replace `YOUR_AGENT_ARN` with your actual agent ARN from deployment.

In [ ]:
# AWS Configuration
AWS_REGION = 'us-east-1'  # Change to your region
AGENT_ARN = "YOUR_AGENT_ARN"  # ⚠️ CHANGE THIS! Get from: agentcore status

# User and Session IDs (persistent across notebook restarts)
USER_ID = "demo-user-alice-12345678901234567890"  # Must be 33+ characters
SESSION_1_ID = "session-1-alice-travel-planning-12345678"  # Must be 33+ characters
SESSION_2_ID = "session-2-alice-follow-up-87654321"  # Must be 33+ characters

print(f"🌍 Region: {AWS_REGION}")
print(f"🤖 Agent ARN: {AGENT_ARN}")
print(f"👤 User ID: {USER_ID}")
print(f"📝 Session 1 ID: {SESSION_1_ID}")
print(f"📝 Session 2 ID: {SESSION_2_ID}")
print(f"\n⚠️  Note: Session and User IDs are persistent - memory will be maintained even if you restart this notebook!")

## Helper Functions

In [ ]:
# Initialize AgentCore client
agentcore_client = boto3.client('bedrock-agentcore', region_name=AWS_REGION)

def invoke_agent(prompt, session_id, user_id, media=None):
    """
    Invoke the AgentCore agent with text and optional media.
    
    Args:
        prompt: Text message to send
        session_id: Session ID (33+ characters)
        user_id: User ID (33+ characters)
        media: Optional dict with 'type', 'format', 'data' (base64)
    
    Returns:
        Agent response as string
    """
    # Prepare payload
    payload_data = {"prompt": prompt}
    if media:
        payload_data["media"] = media
    
    payload = json.dumps(payload_data).encode()
    
    # Invoke agent
    response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=AGENT_ARN,
        runtimeSessionId=session_id,
        runtimeUserId=user_id,
        payload=payload,
        qualifier="DEFAULT"
    )
    
    # Parse response
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    
    if content:
        response_text = ''.join(content)
        try:
            response_json = json.loads(response_text)
            return response_json.get("result", response_text)
        except json.JSONDecodeError:
            return response_text
    
    return "No response from agent"

def print_conversation(role, message, session_id=None):
    """Pretty print conversation"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    session_info = f" [Session: {session_id[:20]}...]" if session_id else ""
    print(f"\n{'='*80}")
    print(f"[{timestamp}] {role}{session_info}")
    print(f"{'='*80}")
    print(message)
    print(f"{'='*80}\n")

print("✅ Helper functions loaded!")

## Session 1: Establishing Travel Preferences

In the first session, we'll share travel preferences with the agent. The agent will store this information in its persistent memory.

In [ ]:
prompt = """Hi! I'm Alice and I'm planning my next trip. Let me share my travel preferences:

- **Food**: I'm vegetarian and love Italian cuisine, especially pasta and risotto
- **Activities**: I enjoy art museums, historical sites, and walking tours
- **Budget**: Around $3000 for a week-long trip
- **Accommodation**: I prefer boutique hotels in central locations
- **Pace**: I like a relaxed pace with time to explore each place

Please remember these preferences for our future conversations!
"""

print_conversation("😊 User (Alice)", prompt, SESSION_1_ID)

response = invoke_agent(prompt, SESSION_1_ID, USER_ID)

print_conversation("🤖 Agent", response, SESSION_1_ID)

### Follow-up in Same Session

Let's ask a follow-up question in the same session to test short-term memory.

In [ ]:
prompt = "Based on my preferences, which European city would you recommend for my next trip?"

print_conversation("😊 User (Alice)", prompt, SESSION_1_ID)

response = invoke_agent(prompt, SESSION_1_ID, USER_ID)

print_conversation("🤖 Agent", response, SESSION_1_ID)

## 🔄 Clear Notebook Context

**Important**: Now we'll simulate a completely new session. Even though we're in the same notebook, we'll use a different session ID. The agent should still remember Alice's preferences because they're stored in persistent memory tied to the USER_ID.

You can even restart the kernel or close this notebook - the memory persists!

## Session 2: Testing Cross-Session Memory

Now we'll use a **different session ID** but the **same user ID**. The agent should remember Alice's preferences from Session 1.

In [ ]:
prompt = """Hi! I'm back. Can you remind me what you know about my travel preferences? 
And based on that, suggest a 3-day itinerary for me."""

print("\n" + "*"*80)
print("🔄 NEW SESSION - Testing Cross-Session Memory")
print("*"*80 + "\n")

print_conversation("😊 User (Alice)", prompt, SESSION_2_ID)

response = invoke_agent(prompt, SESSION_2_ID, USER_ID)

print_conversation("🤖 Agent", response, SESSION_2_ID)

print("\n✅ If the agent remembered your preferences, cross-session memory is working!")

## Multimodal: Sending Images

Let's send an image to the agent for analysis. The agent will analyze it and provide travel recommendations.

In [ ]:
# Path to your image (update this path)
IMAGE_PATH = "data-sample/destination-photo.jpg"  # Change to your image path

# Check if file exists
if os.path.exists(IMAGE_PATH):
    # Read and encode image
    with open(IMAGE_PATH, 'rb') as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
    
    # Determine format
    image_format = IMAGE_PATH.split('.')[-1].lower()
    if image_format == 'jpg':
        image_format = 'jpeg'
    
    prompt = "I found this image of a destination. Can you analyze it and tell me if it matches my travel preferences?"
    
    print_conversation("😊 User (Alice)", f"{prompt}\n[Sending image: {IMAGE_PATH}]", SESSION_2_ID)
    
    # Send image
    media = {
        "type": "image",
        "format": image_format,
        "data": image_data
    }
    
    response = invoke_agent(prompt, SESSION_2_ID, USER_ID, media=media)
    
    print_conversation("🤖 Agent", response, SESSION_2_ID)
else:
    print(f"⚠️  Image not found: {IMAGE_PATH}")
    print("Please update IMAGE_PATH with a valid image file.")

## Multimodal: Sending Videos

Let's send a video to the agent for analysis. Note: Videos must be under 20MB.

In [ ]:
# Path to your video (update this path)
VIDEO_PATH = "data-sample/travel-video.mp4"  # Change to your video path

# Check if file exists
if os.path.exists(VIDEO_PATH):
    # Check file size
    file_size_mb = os.path.getsize(VIDEO_PATH) / (1024 * 1024)
    
    if file_size_mb > 20:
        print(f"⚠️  Video is too large: {file_size_mb:.1f}MB (max 20MB)")
        print("Please use a smaller video or compress it.")
    else:
        print(f"📹 Video size: {file_size_mb:.2f}MB")
        
        # Read and encode video
        with open(VIDEO_PATH, 'rb') as f:
            video_data = base64.b64encode(f.read()).decode('utf-8')
        
        # Determine format
        video_format = VIDEO_PATH.split('.')[-1].lower()
        
        prompt = "I'm sharing a travel video with you. Can you analyze it and suggest similar destinations that match my preferences?"
        
        print_conversation("😊 User (Alice)", f"{prompt}\n[Sending video: {VIDEO_PATH}]", SESSION_2_ID)
        
        # Send video
        media = {
            "type": "video",
            "format": video_format,
            "data": video_data
        }
        
        print("⏳ Analyzing video (this may take a moment)...\n")
        
        response = invoke_agent(prompt, SESSION_2_ID, USER_ID, media=media)
        
        print_conversation("🤖 Agent", response, SESSION_2_ID)
        print("\n📝 Note: Video analysis is visual only (no audio processing)")
else:
    print(f"⚠️  Video not found: {VIDEO_PATH}")
    print("Please update VIDEO_PATH with a valid video file.")

## Session 3: Memory Persistence Test

Let's create yet another session to verify the agent remembers everything, including the image/video we just analyzed.

In [ ]:
SESSION_3_ID = "session-3-alice-final-check-99999999"  # New session ID

prompt = """Can you summarize everything you know about me and my travel preferences? 
Also, what images or videos have I shared with you?"""

print("\n" + "*"*80)
print("🔄 ANOTHER NEW SESSION - Testing Complete Memory Persistence")
print("*"*80 + "\n")

print_conversation("😊 User (Alice)", prompt, SESSION_3_ID)

response = invoke_agent(prompt, SESSION_3_ID, USER_ID)

print_conversation("🤖 Agent", response, SESSION_3_ID)

print("\n✅ The agent should remember:")
print("   - Your travel preferences from Session 1")
print("   - The city recommendation from Session 1")
print("   - The image analysis from Session 2")
print("   - The video analysis from Session 2 (if you sent one)")

## Summary

### What We Demonstrated

1. **Short-term Memory**: Within Session 1, the agent remembered the conversation context
2. **Long-term Memory**: Across Sessions 2 and 3, the agent remembered Alice's preferences
3. **Multimodal Memory**: The agent remembered images and videos shared in previous messages
4. **User Isolation**: All memories are tied to USER_ID, enabling multi-user support
5. **Session Persistence**: Memory persists even if you restart this notebook

### Key Concepts

- **User ID**: Identifies the user across all sessions (must be 33+ characters)
- **Session ID**: Identifies a conversation session (must be 33+ characters)
- **Memory Namespaces**: Agent stores facts and preferences in separate namespaces
- **Cross-Session**: Same user ID + different session IDs = memory persistence

### Next Steps

- Try restarting the kernel and running Session 2 or 3 again - memory persists!
- Test with different user IDs to see memory isolation
- Monitor memory in AWS Console: Bedrock → AgentCore → Memory
- Check agent logs in CloudWatch for debugging

## Optional: Test with Different User

Let's test memory isolation by using a different user ID.

In [ ]:
# Different user
USER_2_ID = "demo-user-bob-98765432109876543210"
SESSION_BOB_ID = "session-bob-first-time-11111111111"

prompt = "Hi! What do you know about my travel preferences?"

print("\n" + "*"*80)
print("👤 DIFFERENT USER - Testing Memory Isolation")
print("*"*80 + "\n")

print_conversation("😊 User (Bob)", prompt, SESSION_BOB_ID)

response = invoke_agent(prompt, SESSION_BOB_ID, USER_2_ID)

print_conversation("🤖 Agent", response, SESSION_BOB_ID)

print("\n✅ The agent should NOT know Bob's preferences (different user ID)")
print("   This demonstrates proper memory isolation between users.")